In [12]:
# Define data paths
personas_path = Path('../data/cuestionario_ampliado/Censo2020_CA_nl_csv/Personas19.CSV')
viviendas_path = Path('../data/cuestionario_ampliado/Censo2020_CA_nl_csv/Viviendas19.CSV')

# Load survey data
personas = process_people_df(personas_path)
viviendas = process_places_df(viviendas_path)

# Select subset of categorical columns to control for
# Seleting them before dropping nan from survey
# Alternative is to impute NAN (e.g. use missforest)
pcat = personas[[
    'ID_PERSONA', 'ID_VIV', 'FACTOR', 'MUN',
    'SEXO', 'EDAD',
    # 'ENT_PAIS_NAC',
    # 'AFRODES',
    'DHSERSAL1', 'DHSERSAL2', 'RELIGION',
    # 'DIS_VER', 'DIS_OIR', 'DIS_CAMINAR', 'DIS',
    # 'DIS_RECORDAR', 'DIS_BANARSE', 'DIS_HABLAR', 'DIS_MENTAL',
    # 'HLENGUA',
    # 'HESPANOL',  # Global seed zero problem 
    'ASISTEN', 'NIVACAD', 'ESCOLARI', 'ALFABET',
    # 'ENT_PAIS_RES_5A',
    'SITUA_CONYUGAL', 'CONACT',
    'INGTRMEN', 'HORTRA'
]].copy()
vcat = viviendas.copy()

# Drop NA values on both surveys

# Look for viviendas with NA values in people and household constraints
na_vivs_v = vcat.ID_VIV[vcat.isna().T.sum() > 0].to_list()
na_vivs_p = pcat.ID_VIV[pcat.isna().T.sum() > 0].to_list()
na_vivs = set(na_vivs_v + na_vivs_p)

# Drop NA before categorizing
pcat = pcat[~pcat.ID_VIV.isin(na_vivs)].reset_index(drop=True)
vcat = vcat[~vcat.ID_VIV.isin(na_vivs)].reset_index(drop=True)

# Categorize columns
pcat = categorize_p(pcat)
vcat = categorize_v(vcat)

assert pcat.isna().sum().sum() == 0
assert vcat.isna().sum().sum() == 0

# Leave only categorized columns
pcat = pcat.drop(columns=[
    'DHSERSAL1', 'DHSERSAL2',
    'NIVACAD', 'ESCOLARI',
    'INGTRMEN', 'HORTRA',
    'DHSERSAL_IMSS', 'DHSERSAL_ISSSTE', 'DHSERSAL_ISSSTE_E',
    'DHSERSAL_P_D_M', 'DHSERSAL_Popular_NGenración_SBienestar',
    'DHSERSAL_IMSS_Prospera/Bienestar'
])

vcat = vcat[[
    'ID_VIV', 'FACTOR', 'MUN', 'NUMPERS',
    'CLAVIVP',
    'AUTOPROP', 'MOTOCICLETA', 'BICICLETA',
    # 'PISOS',
    'CUADORM', 'TOTCUART',
    # 'ELECTRICIDAD', 'AGUA_ENTUBADA',
    # 'ABA_AGUA_ENTU',
    # 'TINACO', 'CISTERNA',
    # 'SERSAN',
    # 'CONAGUA',
    # 'DRENAJE',
    'REFRIGERADOR', 'LAVADORA', 'HORNO',
    'RADIO', 'TELEVISOR',
    'COMPUTADORA', 'TELEFONO', 'CELULAR', 'INTERNET', 'SERV_TV_PAGA',
    'SERV_PEL_PAGA', 'CON_VJUEGOS',
    'JEFE_SEXO'
]]

print(f'We keep {pcat.shape[0]/personas.shape[0]*100}% and {vcat.shape[0]/viviendas.shape[0]*100} of the people and household datasets respectively.')

We keep 95.276327279856% and 95.79107969282408 of the people and household datasets respectively.


In [13]:
pcat

,ID_PERSONA,ID_VIV,FACTOR,MUN,SEXO,EDAD,RELIGION,ASISTEN,ALFABET,SITUA_CONYUGAL,CONACT,DHSERSAL_Privado,DHSERSAL_Otro,DHSERSAL_No afiliado,DHSERSAL_PUB,DHSERSAL_AFIL,EDUC
0,19001000000100001,190010000001,1,Abasolo,F,8-11,Católica,Sí,Sí,Blanco por pase,Blanco por pase,0,0,0,1,1,Primaria_incom
1,19001000000100002,190010000001,1,Abasolo,M,25-49,Católica,No,Sí,casado,Trabaja,0,0,0,1,1,Secundaria_com
2,19001000000100003,190010000001,1,Abasolo,M,0-2,Católica,Blanco por pase,Blanco por pase,Blanco por pase,Blanco por pase,0,0,0,1,1,Blanco por pase
3,19001000000100004,190010000001,1,Abasolo,F,6-7,Católica,No,No,Blanco por pase,Blanco por pase,0,0,0,1,1,Sin Educación
4,19001000000100005,190010000001,1,Abasolo,F,25-49,Católica,No,Sí,casado,No trabaja,0,0,0,1,1,Posbásica
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320758,19051000088400002,190510000884,2,Villaldama,M,12-14,Católica,Sí,Sí,soltero,No trabaja,0,0,0,1,1,Secundaria_incom
320759,19051000088400003,190510000884,2,Villaldama,F,12-14,Católica,Sí,Sí,soltero,No trabaja,0,0,0,1,1,Secundaria_incom
320760,19051000088400004,190510000884,2,Villaldama,M,15-17,Católica,Sí,Sí,soltero,No trabaja,0,0,0,1,1,Posbásica
320761,19051000088400005,190510000884,2,Villaldama,F,25-49,Católica,No,Sí,casado,No trabaja,0,0,0,1,1,Secundaria_com


In [14]:
vcat

,ID_VIV,FACTOR,MUN,NUMPERS,CLAVIVP,AUTOPROP,MOTOCICLETA,BICICLETA,CUADORM,TOTCUART,...,RADIO,TELEVISOR,COMPUTADORA,TELEFONO,CELULAR,INTERNET,SERV_TV_PAGA,SERV_PEL_PAGA,CON_VJUEGOS,JEFE_SEXO
0,190010000001,1,Abasolo,5,Vivienda,No,No,No,1,2,...,Sí,Sí,Sí,No,Sí,No,Sí,No,Sí,F
1,190010000002,1,Abasolo,3,Vivienda,No,No,No,1,2,...,Sí,Sí,No,No,Sí,No,Sí,No,No,M
2,190010000003,1,Abasolo,3,Vivienda,No,No,No,1,3+,...,Sí,Sí,No,No,Sí,No,Sí,No,No,M
3,190010000004,1,Abasolo,3,Vivienda,No,No,Sí,1,2,...,Sí,Sí,No,No,Sí,No,Sí,No,No,M
4,190010000005,1,Abasolo,5,Vivienda,Sí,No,No,2+,3+,...,Sí,Sí,Sí,No,Sí,Sí,No,Sí,Sí,M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94172,190510000880,1,Villaldama,1,Vivienda,No,No,Sí,1,3+,...,No,Sí,No,No,Sí,No,No,No,No,M
94173,190510000881,2,Villaldama,1,Vivienda,No,No,No,1,3+,...,No,No,No,No,No,No,No,No,No,M
94174,190510000882,2,Villaldama,2,Vivienda,Sí,Sí,No,1,2,...,No,No,No,No,No,No,No,No,No,M
94175,190510000883,1,Villaldama,6,Vivienda,Sí,No,No,2+,3+,...,No,No,No,No,No,No,No,No,No,M


In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import scipy
import geopandas as gpd
import xarray as xr
import sparse
from itertools import product, combinations
from pathlib import Path
from pprint import pprint
import matplotlib.pyplot as plt
import matplotlib
from collections import defaultdict

pd.options.display.max_rows = 500
#pd.options.display.max_columns = 4000

import sys
sys.path.append('../src/')

from extended_survey import process_people_df, process_places_df, categorize_p, categorize_v
from census import process_census
from constraints import get_ind_const, get_viv_const

survey_dir = Path('../data/cuestionario_ampliado/Censo2020_CA_nl_csv/')
personas_path = Path('../data/cuestionario_ampliado/Censo2020_CA_nl_csv/Personas19.CSV')
viviendas_path = Path('../data/cuestionario_ampliado/Censo2020_CA_nl_csv/Viviendas19.CSV')
census_iter_path = Path('../data/census_loc/ITER_19CSV20.csv')
census_resageburb_path = Path('../data/census_ageb_manz/RESAGEBURB_19CSV20.csv')
output_path = Path('../output/')

In [3]:
personas = process_people_df(personas_path)
viviendas = process_places_df(viviendas_path)

pcat = personas[[
    'ID_PERSONA', 'ID_VIV', 'FACTOR', 'MUN',
    'SEXO', 'EDAD',
    # 'ENT_PAIS_NAC',
    # 'AFRODES',
    'DHSERSAL1', 'DHSERSAL2', 'RELIGION',
    # 'DIS_VER', 'DIS_OIR', 'DIS_CAMINAR', 'DIS',
    # 'DIS_RECORDAR', 'DIS_BANARSE', 'DIS_HABLAR', 'DIS_MENTAL',
    # 'HLENGUA',
    # 'HESPANOL',  # Global seed zero problem 
    'ASISTEN', 'NIVACAD', 'ESCOLARI', 'ALFABET',
    # 'ENT_PAIS_RES_5A',
    'SITUA_CONYUGAL', 'CONACT',
    'INGTRMEN', 'HORTRA'
]].copy()
vcat = viviendas.copy()

# Look for viviendas with NA
na_vivs_v = vcat.ID_VIV[vcat.isna().T.sum() > 0].to_list()
na_vivs_p = pcat.ID_VIV[pcat.isna().T.sum() > 0].to_list()
na_vivs = set(na_vivs_v + na_vivs_p)

# Drop NA before categorizing
pcat = pcat[~pcat.ID_VIV.isin(na_vivs)].reset_index(drop=True)
vcat = vcat[~vcat.ID_VIV.isin(na_vivs)].reset_index(drop=True)

pcat = categorize_p(pcat)
vcat = categorize_v(vcat)

# Dopr NA again after discretizing
na_vivs = pcat.ID_VIV[pcat.isna().T.sum() > 0].to_list()
pcat = pcat[~pcat.ID_VIV.isin(na_vivs)].reset_index(drop=True)
vcat = vcat[~vcat.ID_VIV.isin(na_vivs)].reset_index(drop=True)

assert pcat.isna().sum().sum() == 0
assert vcat.isna().sum().sum() == 0

# Leave only constrained columns
pcat = pcat[[
    'ID_PERSONA', 'ID_VIV', 'FACTOR',
    'MUN',
    'SEXO', 'EDAD',
    # 'ENT_PAIS_NAC',
    # 'AFRODES',
    #'RELIGION',
    # 'DIS_VER', 'DIS_OIR', 'DIS_CAMINAR',
    # 'DIS_RECORDAR', 'DIS_BANARSE', 'DIS_HABLAR', 'DIS_MENTAL',
    # 'DIS_CON', 'DIS_LIMI',
    # 'HLENGUA',
    # 'HESPANOL',
    'ASISTEN',  'EDUC',
    # 'ALFABET',
    # 'ENT_PAIS_RES_5A',
    'SITUA_CONYUGAL',
    'CONACT',
    # 'DHSERSAL_IMSS', 'DHSERSAL_ISSSTE', 'DHSERSAL_ISSSTE_E', 'DHSERSAL_P_D_M',
    # 'DHSERSAL_Popular_NGenración_SBienestar',
    # 'DHSERSAL_IMSS_Prospera/Bienestar',
    'DHSERSAL_Privado', 'DHSERSAL_Otro',
    'DHSERSAL_No afiliado', 'DHSERSAL_PUB', 'DHSERSAL_AFIL',
]]

vcat = vcat[[
    'ID_VIV', 'FACTOR', 'MUN', 'NUMPERS',
    'CLAVIVP',
    # 'PISOS',
    'CUADORM', 'TOTCUART',
    # 'ELECTRICIDAD', 'AGUA_ENTUBADA',
    # 'ABA_AGUA_ENTU',
    # 'TINACO', 'CISTERNA',
    # 'SERSAN',
    # 'CONAGUA',
    # 'DRENAJE',
    'REFRIGERADOR', 'LAVADORA', 'HORNO',
    'AUTOPROP', 'MOTOCICLETA', 'BICICLETA', 'RADIO', 'TELEVISOR',
    'COMPUTADORA', 'TELEFONO', 'CELULAR', 'INTERNET', 'SERV_TV_PAGA',
    'SERV_PEL_PAGA', 'CON_VJUEGOS',
    'JEFE_SEXO'
]]

print(pcat.shape[0]/personas.shape[0])
print(vcat.shape[0]/viviendas.shape[0])

0.95276327279856
0.9579107969282409


In [4]:
constraints_ind = get_ind_const()
constraints_viv = get_viv_const()

In [5]:
(
    df_mun, df_loc,
    df_agebs, df_agebs_min, df_agebs_max
) = process_census(census_iter_path, census_resageburb_path)

In [5]:
# Build matrices

In [6]:
from setup_lin_system import make_init_system, get_W

In [6]:
X, I, J, L, W, Up, Uh, U, C, Y = make_init_system(pcat, vcat, constraints_ind, constraints_viv, df_mun)

NameError: name 'make_init_system' is not defined

In [8]:
U.columns.name = 'ID_VIV'

In [9]:
X.shape[0]/pcat.shape[0]

0.10308545561676379

In [10]:
C.shape

(51, 63)

In [11]:
mun_list = X.MUN.unique()
const_zeroprob_list = []
for mun in mun_list:
    mun_mask = Y.MUN == mun
    U_mun = U.loc[:, mun_mask]
    const_zeroprob_list.extend(U_mun.index[U_mun.T.sum() == 0].to_list())
set(const_zeroprob_list)

set()

In [12]:
# Reduce the number of variables by identifying equivalent households
# These are equivalent from the perspective of the solver, since they
# contribute equally to the constraints.
# Other attributes may differ, but can be recovered in post-processing and
# assigned accordingly to the sample proportions.
gcols = U.index.to_list()
H = U.T.copy()
H = H.join(Y)
H = H.reset_index().groupby(
    gcols + ['MUN'], observed=True
).agg(
    {'ID_VIV': list} | {c: 'sum' for c in Y.columns.drop('MUN')}
).reset_index().sort_values('MUN').reset_index()
Uh = H[gcols].T
Yh = H[Y.columns]
h_to_y = H.ID_VIV
#Yh = h_to_y.index
print(f'{U.shape[1]} orignal households compress into {Uh.shape[1]} distinct prototypes.')

94177 orignal households compress into 72991 distinct prototypes.


In [13]:
from ortools.sat.python import cp_model

In [14]:
def create_vars(model, Y, min_val, max_val, prefix=''):
    y = {}
    for var_id in Y:
        y[var_id] = model.NewIntVar(min_val, max_val, f'{prefix}{var_id}')
    return y

In [15]:
def create_const(model, C_dict, U, y, relax_factor=1):
    for c_name, c_val in C_dict.items():
        coefficients = U.loc[c_name][U.loc[c_name] > 0]
        expressions = [y[vid] for vid in coefficients.index]
        if c_name in ['POBTOT', 'TVIVHAB']:
            model.Add(cp_model.LinearExpr.WeightedSum(expressions, coefficients.values) == c_val)
        else:
            model.Add(cp_model.LinearExpr.WeightedSum(expressions, coefficients.values) >= relax_factor*c_val)

In [16]:
def solve_sat_all(mun, df_mun, C, Yh, Uh, taz_path='taz_census.gpkg'):
    
    taz_gdf = gpd.read_file(taz_path, layer=mun)
    print(f'{mun} has {taz_gdf.shape[0]} taz')
    
    # Total number of households
    N_mun = int(df_mun.loc[mun, 'TVIVHAB'])

    # Load the constraints as a dictionary
    C_mun = C.loc[mun].astype(int).to_dict()
    C_taz_all = taz_gdf.set_index('ZONA')[C_mun.keys()].fillna(0).astype(int)

    # Get the sample households ids and the contraint weight matrix
    Y_mun = Yh.loc[Yh[mun] > 0, mun]
    U_mun = Uh.loc[:, Y_mun.index]
    assert U_mun.T.duplicated().sum() == 0
    
    model = cp_model.CpModel()
    
    # Create municipality level variables
    y_mun = create_vars(model, Y_mun.index, 1, N_mun)
    
    # Add municipality level constraints
    create_const(model, C_mun, U_mun, y_mun)
    
    # Load TAZ into model
    y_taz = {}
    for taz, C_taz in C_taz_all.iterrows():
        C_taz = C_taz.to_dict()
        N_taz = C_taz['TVIVHAB']
        if N_taz == 0:
            print(f'Ignoring empty taz {taz}.')
            continue
        assert N_taz > 0
    
        # Add variables and constraints for specific taz
        y_taz[taz] = create_vars(model, Y_mun.index, 0, N_taz, f'{taz}_')
        create_const(model, C_taz, U_mun, y_taz[taz], relax_factor=1)
        
    # Create taz to mun hierarchichal constraints
    for var_id, var_mun in y_mun.items():
        var_tazs = []
        for taz_dict in y_taz.values():
            var_tazs.append(taz_dict[var_id])
        model.Add(sum(var_tazs) == var_mun)
    
    # print(model.ModelStats())

    solver = cp_model.CpSolver()

    status = solver.Solve(model)
    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        print('Solution found.')
    else:
        print("No solution found.")
        
    return solver

In [38]:
def solve_sat_single(mun, df_mun, C, Yh, Uh, taz_path='taz_census.gpkg'):
    
    taz_gdf = gpd.read_file(taz_path, layer=mun)
    print(f'{mun} has {taz_gdf.shape[0]} taz')
    
    # Total number of households
    N_mun = int(df_mun.loc[mun, 'TVIVHAB'])

    # Load the constraints as a dictionary
    C_mun = C.loc[mun].astype(int).to_dict()
    C_taz_all = taz_gdf.set_index('ZONA')[C_mun.keys()].fillna(0).astype(int)

    # Get the sample households ids and the contraint weight matrix
    Y_mun = Yh.loc[Yh[mun] > 0, mun]
    U_mun = Uh.loc[:, Y_mun.index]
    assert U_mun.T.duplicated().sum() == 0
    
    # Load TAZ into model
    for taz, C_taz in C_taz_all.iterrows():
        if taz == -10: continue

        C_taz = C_taz.to_dict()
        N_taz = C_taz['TVIVHAB']
        if N_taz == 0:
            print(f'Ignoring empty taz {taz}.')
            continue
        assert N_taz > 0

        model = cp_model.CpModel()
             
        # Add variables and constraints for specific taz
        y_taz = create_vars(model, Y_mun.index, 0, N_taz, '')
        create_const(model, C_taz, U_mun, y_taz, relax_factor=1)
        
        solver = cp_model.CpSolver()
        
        return solver, model, y_taz, Y_mun, C_taz, C_mun
        
        solution_printer = VarArraySolutionPrinter()
        # Enumerate all solutions.
        solver.parameters.enumerate_all_solutions = True
        # Solve.
        status = solver.Solve(model, solution_printer)
        # status = solver.Solve(model)

        if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
            print(f'Solution found for {taz}.')
        else:
            print(f"No solution found for {taz}.")
            
        return y_taz, solver, U_mun, C_taz, solution_printer

In [92]:
solver, model, y_taz, Y_mun, C_taz, C_mun = solve_sat_single('Monterrey', df_mun, C, Yh, Uh)

Monterrey has 89 taz


In [30]:
print(model.ModelStats())

satisfaction model '': (model_fingerprint: 0xdaa44dfbb84a649d)
#Variables: 3'296
  - 3'296 in [0,2406]
#kLinearN: 63 (#terms: 99'996)


In [67]:
print(model.ModelStats())

satisfaction model '': (model_fingerprint: 0x821976279636e877)
#Variables: 6'592
  - 3'296 in [0,2406]
  - 3'296 in [0,726944028]
#kLinMax: 3'296 (#expressions: 6'592)
#kLinearN: 63 (#terms: 99'996)


In [32]:
len(y_taz)

3296

In [40]:
Y_mun.sum()

302138

In [42]:
C_taz['TVIVHAB']

2406

In [45]:
C_mun['TVIVHAB']

329095

In [49]:
round(302138/2406)*2406

303156

In [51]:
Ns = Y_mun.sum() # Total H in survey
Nt = C_taz['TVIVHAB']  # Total H in TAZ

In [54]:
Ns/Nt

125.57689110556942

In [65]:
Y_mun

53257     65
53258     78
53259    132
53260    125
53261     60
        ... 
56548     56
56549     64
56550     60
56551     62
56552     56
Name: Monterrey, Length: 3296, dtype: int64

In [62]:
y_taz[53257]

53257(0..2406)

In [93]:
# Create abs val intermediate variables
abs_vars = {}
max_abs = Ns*Nt
for var_id, var_taz in y_taz.items():
    abs_vars[var_id] = model.NewIntVar(0, max_abs, f'abs_{var_id}')
    model.AddAbsEquality(abs_vars[var_id], Ns*y_taz[var_id] - Nt*Y_mun.loc[var_id])

In [78]:
# Create abs val intermediate variables
abs_vars = {}
factor = round(Ns/Nt)
max_abs = Ns
for var_id, var_taz in y_taz.items():
    abs_vars[var_id] = model.NewIntVar(0, max_abs, f'abs_{var_id}')
    model.AddAbsEquality(abs_vars[var_id], factor*y_taz[var_id] - Y_mun.loc[var_id])

In [94]:
# Create objective function
model.Minimize(sum(abs_vars.values()))

In [100]:
solver.parameters.relative_gap_limit = 0.01

In [ ]:
%%time
status = solver.Solve(model)
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print(f'Solution found.')
else:
    print(f"No solution found.")

In [97]:
solver.BestObjectiveBound()

451669418.0

In [99]:
solver.ObjectiveValue()

197469.7123857024

In [90]:
len(y_taz)

3296

In [91]:
196820/3296

59.71480582524272

In [22]:
class VarArraySolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self):
        cp_model.CpSolverSolutionCallback.__init__(self)
        # self.__variables = variables
        self.__solution_count = 0

    def on_solution_callback(self):
        self.__solution_count += 1
        #for v in self.__variables:
        #    print(f"{v}={self.Value(v)}", end=" ")
        #print()

    def solution_count(self):
        return self.__solution_count

In [248]:
%%time
y_taz, solver, U_mun, C_taz, printer = solve_sat_single('Monterrey', df_mun, C, Yh, Uh)
printer.solution_count()

Monterrey has 89 taz
Solution found for 1.
CPU times: user 18min 16s, sys: 10.2 s, total: 18min 26s
Wall time: 18min 26s


1409

In [249]:
y_sol = solver.Values(pd.Series(y_taz))

In [250]:
U_mun.values @ y_sol.values

array([2406, 1316, 1090,  395,  846, 6154, 2987, 3167,  215,  168,   47,
       5939, 3120, 2819, 5873, 3103, 2770, 5420, 2749, 2671, 5336, 2711,
       2625, 4684, 2660, 2024,  110,   61,   49,  409,  310,   99,  472,
        347,  125,   84,   38,   46,  652,   51,  601,  962,  273,  689,
       1011,  465,  546,   66,   13,  577,  230,   27,   41,  437,  275,
        733, 3260, 3632, 1482, 2150, 1788, 1189,  599])

In [213]:
viv_ids = []
for l in h_to_y[y_sol[y_sol > 0].index].values:
    viv_ids.extend(l)

In [214]:
vivtemp = viviendas.set_index('ID_VIV').loc[viv_ids]

In [215]:
pd.crosstab(vivtemp.MOTOCICLETA, vivtemp.AUTOPROP)

AUTOPROP,Sí,No
MOTOCICLETA,,
Sí,1,1
No,14,12


In [232]:
(Y['Monterrey'] * 2406 / Y['Monterrey'].sum())[Y['Monterrey'] > 0]

ID_VIV
190390000001    0.541501
190390000002    0.541501
190390000003    0.541501
190390000004    0.541501
190390000005    0.541501
                  ...   
190390005155    0.015926
190390005156    0.015926
190390005157    0.015926
190390005158    0.015926
190390005159    0.015926
Name: Monterrey, Length: 4794, dtype: float64

In [180]:
solver.Values(pd.Series(y_taz))

53257    0
53258    0
53259    0
53260    0
53261    0
        ..
56548    0
56549    0
56550    0
56551    0
56552    0
Length: 3296, dtype: int64